python setup.py sdist bdist_wheel

twine upload --skip-existing dist/*

In [1]:
import pandas as pd
import ScraperFC as sfc
import traceback

In [2]:
scraper = sfc.WhoScored()
try:
    out = scraper.scrape_matches(year=2022, league='EFL2')
except:
    traceback.print_exc()
finally:
    scraper.close()

Traceback (most recent call last):
  File "C:\Users\Owner\Documents\GitHub\ScraperFC\code\ScraperFC\WhoScored.py", line 193, in scrape_matches
    match_data[link] = self.scrape_match(link)
  File "C:\Users\Owner\Documents\GitHub\ScraperFC\code\ScraperFC\WhoScored.py", line 211, in scrape_match
    scripts.append(el.get_attribute('innerHTML'))
  File "C:\Users\Owner\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py", line 154, in get_attribute
    attribute_value = self.parent.execute_script(
  File "C:\Users\Owner\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 872, in execute_script
    return self.execute(command, {
  File "C:\Users\Owner\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 416, in execute
    response = self.command_executor.execute(driver_command, params)
  File "C:\Users\Owner\anaconda3\lib\site-packages\selenium\webdriver\remote\remote_connection.py", line 398, in execute
    return self._request(c

MaxRetryError: HTTPConnectionPool(host='localhost', port=57973): Max retries exceeded with url: /session/dfd4b488222676065f3c42be0de0a801/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001606883C250>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
print(out[list(out.keys())[-1]])